In [6]:
import os
import pandas as pd
import fasttext
import numpy as np
import time

### Training
- Orthogonal transformation acts on source matrix (or the transpose acts on the target matrix).
- Currently `en` is treated as source and `es` is treated as target. 

In [ ]:
def apply_transform(vec, transform):
    """
    Apply the given transformation to the vector space

    Right-multiplies given transform with embeddings E:
        E = E * transform

    Transform can either be a string with a filename to a
    text file containing a ndarray (compat. with np.loadtxt)
    or a numpy ndarray.
    """
    transmat = np.loadtxt(transform)# if isinstance(transform, str) else transform
    return np.matmul(vec, transmat)

In [ ]:
def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []
    
    len_bd = len(bilingual_dictionary)

    for i, (source, target) in enumerate(bilingual_dictionary):
        print(f'\r{i + 1}/{len_bd} | {100 * (i + 1) / len_bd:.3f} %', end = '', flush = True)
        sourceVector = source_dictionary.get_sentence_vector(source.lower().strip().replace('_',' '))
        targetVector = target_dictionary.get_sentence_vector(target.lower().strip().replace('_',' '))
        source_matrix.append(sourceVector)
        target_matrix.append(targetVector)
        
    # return training matrices
    return np.array(source_matrix), np.array(target_matrix)

In [ ]:
def normalized(a, axis=-1, order=2):
    """Utility function to normalize the rows of a numpy array."""
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

In [ ]:
def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
    """
    Source and target matrices are numpy arrays, shape
    (dictionary_length, embedding_dimension). These contain paired
    word vectors from the bilingual dictionary.
    """
    # optionally normalize the training vectors
    if normalize_vectors:
        source_matrix = normalized(source_matrix)
        target_matrix = normalized(target_matrix)

    # perform the SVD
    product = np.matmul(target_matrix.transpose(), source_matrix)
    U, s, V = np.linalg.svd(product)

    # return orthogonal transformation which aligns source language to the target
    return np.matmul(U, V)

In [ ]:
def apply_transform(vec, transform):
    """
    Apply the given transformation to the vector space

    Right-multiplies given transform with embeddings E:
        E = E * transform

    Transform can either be a string with a filename to a
    text file containing a ndarray (compat. with np.loadtxt)
    or a numpy ndarray.
    """
    transmat = np.loadtxt(transform)# if isinstance(transform, str) else transform
    return np.matmul(vec, transmat)

## Modify depending on your directory structure

In [ ]:
model_path = os.path.join('embeddings', 'fasttext')
sentences_path = os.path.join('sentences', 'wikimatrix')

In [ ]:
model = {}

model['en'] = fasttext.load_model(os.path.join(model_path, 'wiki.en.bin'))
model['es'] = fasttext.load_model(os.path.join(model_path, 'wiki.es.bin'))

### WikiMatrix

In [ ]:
if False:
    with open(os.path.join(sentences_path, 'WikiMatrix.en-es.txt.en'), 'r') as f:
        en = f.read()

    with open(os.path.join(sentences_path, 'WikiMatrix.en-es.txt.es'), 'r') as f:
        es = f.read()

    df = pd.DataFrame(zip(en.split('\n')[:-1], es.split('\n')[:-1]), columns = ['en', 'es'])
    df.head()

### Content Translation

In [4]:
import json
import unicodedata

In [7]:
with open(os.path.join('data', 'contenttranslation', 'cx-corpora.en2es.text.json'), 'r') as f:
    json_data = json.load(f)

In [8]:
json_data[3]['target']['content']

'Laura García Benitez (nacida en 19 abril 1981 en Pamplona) es una judoka\xa0español que ha representado a España en los Juegos Paralímpicos de Verano de 2008 y 2012.'

In [11]:
unicodedata.normalize('NFKC', json_data[3]['target']['content'])

'Laura García Benitez (nacida en 19 abril 1981 en Pamplona) es una judoka español que ha representado a España en los Juegos Paralímpicos de Verano de 2008 y 2012.'

In [ ]:
data = []
for line in json_data:
    try:
        source = unicodedata.normalize('NFKC', line['source']['content'])
        target = unicodedata.normalize('NFKC', line['target']['content'])
        data.append([source, target])
    except TypeError:
        pass

In [ ]:
bilingual_dictionary = data

In [ ]:
source_matrix, target_matrix = make_training_matrices(model['en'], model['es'], bilingual_dictionary)

In [ ]:
if False:
    np.savetxt(os.path.join(sentences_path, 'source.en.txt'), source_matrix)
    np.savetxt(os.path.join(sentences_path, 'target.es.txt'), target_matrix)

In [ ]:
if False:
    source_matrix = np.loadtxt(os.path.join(sentences_path, 'source.en.txt'))
    target_matrix = np.loadtxt(os.path.join(sentences_path, 'target.es.txt'))

In [ ]:
transform = learn_transformation(source_matrix, target_matrix)

### Mean cosine similarity before transformation

In [ ]:
np.matmul(target_matrix[:10000, :10000], source_matrix[:10000, :10000].T).mean()

### Mean cosine similarity after transformation

In [ ]:
np.matmul(target_matrix[:10000, :10000], np.matmul(transform, source_matrix[:10000, :10000].T)).mean()